In [7]:
import linktransformer as lt


/home/njui/kn_workspace/curriculum_taxonomy_extractor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd

In [5]:
# folder_path = "/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/processed/CBC Math/Grade 4 Curriculum designs Maths CSV"

In [32]:
import os
import chardet
import pandas as pd

# Specify multiple folder paths here
folder_paths = [
    "/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/processed/CBC Math/Grade 3 Curriculum designs Maths CSV",
    "/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/processed/CBC Math/Grade 4 Curriculum designs Maths CSV",
    "/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/processed/CBC Math/Grade 6 Curriculum designs Maths CSV",
    # Add more paths as needed
]

# Initialize lists to store the data
file_names = []
file_paths = []
folder_names = []
file_details = []
indicator_files = []
read_status= []

# Keywords to check in the CSV contents
key_words = ["meets expectations", "below expectations"]

# Iterate through each specified folder path
for folder_path in folder_paths:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Construct the full file path
            full_file_path = os.path.join(root, file)
            file_names.append(file)
            file_paths.append(full_file_path)
            folder_name = os.path.basename(root)
            folder_names.append(folder_name)

            # Read the CSV file and convert it to a JSON string
            try:
                with open(full_file_path, 'rb') as file:
                    result = chardet.detect(file.read())
                    encoding = result['encoding']
                df_temp = pd.read_csv(full_file_path, encoding=encoding)
                details = df_temp.to_json()
                file_details.append(details)
                
                # Check if any of the key words are in the CSV
                indicator = any(word in df_temp.to_string().lower() for word in key_words)
                indicator_files.append(indicator)
                read_status.append(True)
            except Exception as e:
                file_details.append({'error': str(e)})
                read_status.append(False)
                indicator_files.append('')  # Default to False in case of error

# Create a dataframe with the collected data
df = pd.DataFrame({
    'file_name': file_names,
    'file_path': file_paths,
    'folder_name': folder_names,
    'details': file_details,
    'indicator_status': indicator_files,
    'read_status': read_status
})


In [33]:
distinct_read_status = df['read_status'].unique()
print(distinct_read_status)

[ True]


In [29]:
# # Set your OpenAI API key
# openai_key = os.getenv('OPENAI_API_KEY')

# # Specify the model you want to use for generating embeddings or similarity scores
# model = "all-MiniLM-L6-v2"

# # Specify the clustering type and parameters
# cluster_type = "SLINK"
# cluster_params = {'threshold': 0.90, "min cluster size": 2, "metric": "cosine"}
# # cluster_params = {
# #     "max cluster size": 2,
# #     "threshold": 0.95,
# #     "max samples": 2  # Default is set to the same as "min cluster size" unless specified.
# # }

# # Call the cluster_rows function, specifying the column to cluster on
# clustered_df = lt.cluster_rows(
#     df=df,
#     model=model,
#     on=['file_name','folder_name'],  # The column to deduplicate on
#     cluster_type=cluster_type,
#     cluster_params=cluster_params,
#     openai_key=openai_key
# )

No base organization specified, if there is an error, it is likely because of that.
Trying to append the model : sentence-transformers/all-MiniLM-L6-v2 and linktransformers/all-MiniLM-L6-v2. Check your path otherwise!
Trying sentence-transformers/all-MiniLM-L6-v2...


2024-03-05 10:15:27 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 10:15:28 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [36]:
df_grouped = df.groupby('indicator_status')

# Get the DataFrames for each group
df_rubrics = df_grouped.get_group(True)
df_strands = df_grouped.get_group(False)


In [40]:
# df_strands.head

In [37]:
# Using the lt.merge function to perform a semantic merge
df_merged = lt.merge(
    df1=df_strands, 
    df2=df_rubrics, 
    on=['file_name','folder_name','details'], 
    merge_type='1:1',
    model='all-MiniLM-L6-v2',
    left_on=None, 
    right_on=None, 
    suffixes=('_x', '_y'), 
    use_gpu=False, 
    batch_size=128, 
    openai_key=os.getenv('OPENAI_API_KEY')  # Assuming you've set your API key in the environment
)


No base organization specified, if there is an error, it is likely because of that.
Trying to append the model : sentence-transformers/all-MiniLM-L6-v2 and linktransformers/all-MiniLM-L6-v2. Check your path otherwise!


Trying sentence-transformers/all-MiniLM-L6-v2...
No base organization specified, if there is an error, it is likely because of that.
Trying to append the model : sentence-transformers/all-MiniLM-L6-v2 and linktransformers/all-MiniLM-L6-v2. Check your path otherwise!
Trying sentence-transformers/all-MiniLM-L6-v2...
2024-03-05 13:48:44 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 13:48:45 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 13:48:45 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 13:48:46 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 13:48:46 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 13:48:46 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 13:48:47 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 1

In [ ]:
df_merged.sort_values(by='score', ascending=False).head(10)

In [41]:
# df_merged.shape

(48, 15)

In [42]:
# df.shape

(96, 6)